In [ ]:
# Install packages
!pip install datasets

# Import packages
from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
from google.colab import drive
import gensim
import numpy as np

import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from sklearn.metrics import accuracy_score # Accuracy 측정 함수 import

##### konlpy 다운로드, 코드 상단에서 KoNLPy, Mecab 설치 명령어 실행
# connect google drive
drive.mount('/content/drive')

# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh
from konlpy.tag import Mecab  # KoNLPy를 통해 Mecab 패키지 import

In [ ]:
##### 데이터 다운로드
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

In [ ]:
##### Word embedding 확인
# load word2vec embedding
embedding_model = gensim.models.Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/word2vec/word2vec')
emb_words = embedding_model.wv.index_to_key

print(emb_words[:10])

['1', '.', '0', '2', '의', ',', '이', '는', '다', ')']


In [ ]:
#@title
##### 데이터 구조 훑어보기
# print("Data type: ", type(data))    # 데이터 타입 확인
# print("Data structure: ", data)     # 데이터 구조 확인
# print("Data keys: ", data.keys())   # 데이터 키 확인

print(data['train'][0])   # 실제 데이터 확인

{'label': 1, 'text': '역시 명작 어렸을때 봤을때도 재밌었고 지금 봐도 몇억배 이상으로 재밌어요'}


In [ ]:
##### 테스트 세트 만들기
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#print(train_data)
#print(dev_data)
#print(test_data)

# mecab 형태소 분석기 사용
mecab = Mecab()

# 형태소 단위로 Tokenization 된 텍스트를 {train/dev/test}_data에 저장
train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})

print(train_data[0])

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}


In [ ]:
##### 임베딩 사용하기
### 룩업테이블 정의
emb_words = embedding_model.wv.index_to_key
emb_vectors = embedding_model.wv.vectors
emb_dim = embedding_model.vector_size
emb_stoi = {}
max_seq_len = 30

# <pad> -> 0, <unk> -> 1
emb_stoi['<pad>'] = 0
emb_stoi['<unk>'] = 1

print(len(emb_vectors))
pad_vector = np.zeros(shape=(1, emb_dim))
unk_vector = np.zeros(shape=(1, emb_dim))
emb_vectors = np.concatenate((pad_vector, unk_vector, emb_vectors), axis=0)
print(len(emb_vectors))

for i, word in enumerate(emb_words):
  emb_stoi[word] = i + 2

emb_vectors = torch.FloatTensor(emb_vectors)


### 인덱스 변환
def text_to_index(input_data, stoi, max_seq_len):
  all_index = []

  for sample in input_data:
    index_list = []
    for word in sample['text'].split():
      if word in stoi.keys(): # 단어가 lookup table에 검색 가능한 경우
        index_list.append(stoi[word])
      else: # 단어가 lookup table에 검색 불가능한 경우
        index_list.append(stoi['<unk>'])

    # Padding: 샘플의 길이가 고정 크기의 최대 길이 (L)보다 작은 경우, pad 인덱스를 추가
    if max_seq_len > len(index_list):
      index_list = index_list + [stoi['<pad>']] * (max_seq_len - len(index_list))
    else: # 샘플의 길이가 고정 크기의 최대 길이 (L)보다 큰 경우, 고정 크기까지만 처리
      index_list = index_list[:max_seq_len]
    all_index.append(index_list)

  return all_index

# train, dev, test data들을 index로 변환
train_index = text_to_index(train_data, emb_stoi, max_seq_len)
dev_index = text_to_index(dev_data, emb_stoi, max_seq_len)
test_index = text_to_index(test_data, emb_stoi, max_seq_len)

# 강의자료 page 20 구현 후 출력 결과 확인
# print(train_index[0])
# print(emb_words[717-2], emb_words[4526-2], emb_words[5861-2], emb_words[16-2],emb_words[100-2])
# print(train_index[100])

358043
358045


In [ ]:
##### 훈련세트에서 훈련하고 평가하기

# randomness 제거
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

class LSTMClassifier(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LSTMClassifier, self).__init__()
    # 1. 임베딩 레이어 선언
    self.word_embedding = nn.Embedding.from_pretrained(embeddings=emb_vectors)
    # 2. LSTM 레이어 선언
    self.lstm = nn.LSTM(input_size = input_size, hidden_size = hidden_size, batch_first=True)
    # 3. 출력 레이어 선언
    self.linear = nn.Linear(hidden_size, output_size)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    # 1. inputs에 대한 임베딩 정보를 읽음
    embedded_sents = self.word_embedding(torch.cuda.LongTensor(x))
    # 2. 임베딩 결과를 LSTM에 입력 # => 처리할 수 있도록 정제하는 과정
    _, (last_hidden, last_cell)= self.lstm(embedded_sents)
    # 3. 마지막 hidden state를 가져옴
    last_hidden = last_hidden.squeeze() # last_hidden[-1] 스퀴즈의 의미를 알아야 한다.
    # 4. LSTM representation (hidden state를 classification label의 차원의 확률로 변경)
    output = self.linear(last_hidden)
    output_prob = self.softmax(output)
    return output_prob


# MLP 모델의 아키텍쳐(구조) 정의
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, max_seq_len):
    super(MLP, self).__init__()
    # embedding layer
    self.word_embedding = nn.Embedding.from_pretrained(embeddings=emb_vectors)
    self.input_size = input_size
    self.max_seq_len = max_seq_len

    # 각 층에 대한 정의
    self.layer1 = nn.Linear(max_seq_len * input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    self.layer3 = nn.Linear(int(0.5*hidden_size), output_size)
    # 활성화 함수
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    # 실제(입력) 데이터: x
    # x를 가지고 순전파를 진행
    word_emb = self.word_embedding(torch.cuda.LongTensor(x))
    word_emb_new = word_emb.view((-1, self.max_seq_len * self.input_size))

    out1 = self.layer1(word_emb_new) #(sum)
    out2 = self.gelu(out1) # hidden_size
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.softmax(out5)

    return out6

'''
class MLP(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(MLP, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.layer2 = nn.Linear(hidden_size, int(0.5*hidden_size))
    self.layer3 = nn.Linear(int(0.5*hidden_size), int(2*hidden_size))
    self.layer4 = nn.Linear(int(2*hidden_size), hidden_size)
    self.layer5 = nn.Linear(hidden_size, output_size)
    self.gelu = nn.GELU()
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    out1 = self.layer1(x)
    out2 = self.gelu(out1)
    out3 = self.layer2(out2)
    out4 = self.gelu(out3)
    out5 = self.layer3(out4)
    out6 = self.gelu(out5)
    out7 = self.layer4(out6)
    out8 = self.gelu(out7)
    out9 = self.layer5(out8)
    out10 = self.softmax(out9)

    return out10
'''

# 하이퍼파라미터 셋팅
input_size = emb_dim #len(vectorizer.vocabulary_) # input size
hidden_size = 100 # hidden size
output_size = 2 # output size is 2 (positive/negative)
#learning_rate = 0.00001
learning_rate = 0.001
batch_size = 128
num_epochs = 20

# 모델 초기화
# model = MLP(input_size, hidden_size, output_size, max_seq_len)
model = LSTMClassifier(input_size, hidden_size, output_size)
device = torch.device("cuda") # use GPU
model = model.to(device)

# optimizer, loss function 정의
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

# dev_vectors와 dev_data['label']을 텐서 자료형으로 변환
dev_tensors = dev_index #torch.cuda.FloatTensor(dev_vectors.toarray(), device=device)
dev_labels = torch.tensor(dev_data['label'], dtype=torch.long, device=device)

# 학습 시작 (총 num_epochs 만큼)
for epoch in range(num_epochs):
  # model을 학습모드로 만든다.
  model.train()

  # 학습을 하면서 보고싶은 숫자 (학습이 잘되는지 확인)
  epoch_loss = 0
  best_accuracy = 0

  # train_vectors를 텐서 자료형으로 변환
  train_tensors = train_index #torch.cuda.FloatTensor(train_vectors.toarray(), device=device)

  # batch size 단위로 학습 진행
  for i in range(0, len(train_tensors), batch_size):

    # batch 단위 데이터 생성
    batch_data = train_tensors[i:i+batch_size] # batch size 크기의 데이터가 batch_data
    batch_labels = torch.tensor(train_data['label'][i:i+batch_size], device=device)

    # 1. 순전파
    outputs = model(batch_data)
    # 2. 오차 계산
    loss = loss_function(outputs, batch_labels)
    # 3. 역전파
    optimizer.zero_grad()
    # 4. 가중치 업데이트
    loss.backward()
    optimizer.step()

    # epoch loss
    epoch_loss += loss.item()

  # 매 epoch마다 dev 성능 측정
  # 모델을 평가하는 모드로 셋팅
  model.eval()
  with torch.no_grad():
    dev_outputs = model(dev_tensors) # dev 데이터의 순전파
    dev_preds = torch.argmax(dev_outputs, axis=1)
    dev_accuracy = torch.sum(dev_preds == dev_labels).item() / len(dev_labels)

    # save best model on dev data
    if dev_accuracy > best_accuracy:
      best_model = model
      best_accuracy = dev_accuracy

  print(f"Epoch {epoch+1}, Accuracy: {dev_accuracy} , loss: {epoch_loss/len(train_tensors)}")

Epoch 1, Accuracy: 0.7336834208552138 , loss: 0.004682156916293833
Epoch 2, Accuracy: 0.7254313578394599 , loss: 0.004202281775573889
Epoch 3, Accuracy: 0.7629407351837959 , loss: 0.004160785536799166
Epoch 4, Accuracy: 0.7606901725431358 , loss: 0.004063790457944075
Epoch 5, Accuracy: 0.7591897974493623 , loss: 0.004034513021508853
Epoch 6, Accuracy: 0.7854463615903976 , loss: 0.003965085103280015
Epoch 7, Accuracy: 0.7764441110277569 , loss: 0.00393780640926626
Epoch 8, Accuracy: 0.8049512378094523 , loss: 0.0038585381590657765
Epoch 9, Accuracy: 0.7779444861215303 , loss: 0.0037964582724703683
Epoch 10, Accuracy: 0.6774193548387096 , loss: 0.0037877341748939622
Epoch 11, Accuracy: 0.800450112528132 , loss: 0.0037909124071399372
Epoch 12, Accuracy: 0.7914478619654913 , loss: 0.0037241879345642197
Epoch 13, Accuracy: 0.8034508627156789 , loss: 0.0037024641161163647
Epoch 14, Accuracy: 0.8012003000750187 , loss: 0.0036983016116751565
Epoch 15, Accuracy: 0.8094523630907727 , loss: 0.003

In [ ]:
##### 테스트 세트로 시스템 평가하기
final_model = best_model # MLP 모델
test_tensors = test_index #torch.cuda.FloatTensor(test_vectors.toarray(), device=device)
test_outputs = final_model(test_tensors) # 최종 모델로 test 데이터 예측
pred_results = torch.argmax(test_outputs, axis=1)
accuracy = accuracy_score(test_data['label'], pred_results.tolist()) # 정확도 측정
print("Accuracy: {:.2f}%".format(accuracy*100)) # 정확도 출력

Accuracy: 75.74%
